In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import tqdm
import os
import wandb

ImportError: cannot import name 'TypeAliasType' from 'typing_extensions' (C:\Users\elias\Documents\Anaconda\envs\ADL\lib\site-packages\typing_extensions.py)

In [ ]:
wandb.login()

wandb.init(project="amazon-books-reviews-sentiment", name="T6-6class", reinit=True)

In [15]:
# Hyperparameters
mb_size = 64
Z_dim = 1000
h_dim = 128
c_dim = 10 # class 
lr = 1e-3

In [16]:
# Load MNIST data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten the 28x28 image to 784
])

train_dataset = datasets.MNIST(root='../MNIST', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=mb_size, shuffle=True)

X_dim = 784  # 28 x 28

In [17]:
# Xavier Initialization
def xavier_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


In [18]:
# To onehot
def to_onehot(x, num_classes=10):
    if isinstance(x, int):
        x = torch.tensor([x], dtype=torch.long)
    elif not isinstance(x, torch.Tensor):
        raise TypeError("Input must be an int or a torch.Tensor")

    x = x.long()  # ensure it's LongTensor for one-hot indexing
    one_hot = torch.zeros(x.size(0), num_classes, device=x.device)
    one_hot.scatter_(1, x.view(-1, 1), 1)
    return one_hot

In [19]:
# Generator
class Generator(nn.Module):
    def __init__(self, z_dim, h_dim, x_dim, c_dim):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(z_dim + c_dim, h_dim)
        self.fc2 = nn.Linear(h_dim, x_dim)
        self.apply(xavier_init)

    def forward(self, z,c):
        v = torch.cat((z, c), 1)
        h = F.relu(self.fc1(v))
        out = torch.sigmoid(self.fc2(h))
        return out


In [20]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self, x_dim, h_dim, c_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(x_dim + c_dim, h_dim)
        self.fc2 = nn.Linear(h_dim, 1)
        self.apply(xavier_init)

    def forward(self, x, c):
        v = torch.cat((x, c), 1)
        h = F.relu(self.fc1(v))
        out = torch.sigmoid(self.fc2(h))
        return out


In [32]:
# Training
def cGANTraining(G, D, loss_fn, train_loader):
    G.train()
    D.train()

    D_loss_real_total = 0
    D_loss_fake_total = 0
    G_loss_total = 0
    t = tqdm.tqdm(train_loader)
    
    for it, (X_real, labels) in enumerate(t):
        # Prepare real data
        X_real = X_real.float().to(device)
        c = to_onehot(labels).to(device)
        
        # Sample noise and labels
        z = torch.randn(X_real.size(0), Z_dim).to(device)
        ones_label = torch.ones(X_real.size(0), 1).to(device)
        zeros_label = torch.zeros(X_real.size(0), 1).to(device)

        # ================= Train Discriminator =================
        G_sample = G(z,c)
        D_real = D(X_real, c)
        D_fake = D(G_sample.detach(),c)

        D_loss_real = loss_fn(D_real, ones_label)
        D_loss_fake = loss_fn(D_fake, zeros_label)
        D_loss = D_loss_real + D_loss_fake
        D_loss_real_total += D_loss_real.item()
        D_loss_fake_total += D_loss_fake.item()

        D_solver.zero_grad()
        D_loss.backward()
        D_solver.step()

        # ================= Train Generator ====================
        z = torch.randn(X_real.size(0), Z_dim).to(device)
        G_sample = G(z,c)
        D_fake = D(G_sample,c)

        G_loss = loss_fn(D_fake, ones_label)
        G_loss_total += G_loss.item()

        G_solver.zero_grad()
        G_loss.backward()
        G_solver.step()

    # ================= Logging =================
    D_loss_real_avg = D_loss_real_total / len(train_loader)
    D_loss_fake_avg = D_loss_fake_total / len(train_loader)
    D_loss_avg = D_loss_real_avg + D_loss_fake_avg
    G_loss_avg = G_loss_total / len(train_loader)

#    wandb.log({
#        "D_loss_real": D_loss_real_avg,
#        "D_loss_fake": D_loss_fake_avg,
#        "D_loss": D_loss_avg,
#        "G_loss": G_loss_avg
#    })

    return G, D, G_loss_avg, D_loss_avg
    


def save_sample(G, epoch, mb_size, Z_dim):
    out_dir = "out_vanila_GAN2"
    G.eval()
    with torch.no_grad():
        num_classes = 10
        z = torch.randn(num_classes, Z_dim).to(device)

        # One-hot encoded class conditions
        c = torch.zeros(num_classes, num_classes).to(device)
        c[torch.arange(num_classes), torch.arange(num_classes)] = 1

        samples = G(z, c).detach().cpu().numpy()

    fig = plt.figure(figsize=(10, 1))
    for i in range(num_classes):
        plt.subplot(1, num_classes, i + 1)
        plt.axis('off')
        plt.imshow(samples[i].reshape(28, 28), cmap='Greys_r')

    os.makedirs(out_dir, exist_ok=True)
    plt.savefig(f'{out_dir}/{str(epoch).zfill(3)}.png', bbox_inches='tight')
    plt.close(fig)



########################### Main #######################################
wandb_log = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Instantiate models
G = Generator(Z_dim, h_dim, X_dim, c_dim).to(device)
D = Discriminator(X_dim, h_dim, c_dim).to(device)

# Optimizers
G_solver = optim.Adam(G.parameters(), lr=lr)
D_solver = optim.Adam(D.parameters(), lr=lr)

# Loss function
def my_bce_loss(preds, targets):
    return F.binary_cross_entropy(preds, targets)

#loss_fn = nn.BCEWithLogitsLoss()
loss_fn = my_bce_loss

#if wandb_log: 
#    wandb.init(project="conditional-gan-mnist")
#
#    # Log hyperparameters
#    wandb.config.update({
#        "batch_size": mb_size,
#        "Z_dim": Z_dim,
#        "X_dim": X_dim,
#        "h_dim": h_dim,
#        "lr": lr,
#    })
#
best_g_loss = float('inf')  # Initialize best generator loss
save_dir = 'checkpoints'
os.makedirs(save_dir, exist_ok=True)

#Train epochs
epochs = 100

for epoch in range(epochs):
    G, D, G_loss_avg, D_loss_avg= cGANTraining(G, D, loss_fn, train_loader)

    print(f'epoch{epoch}; D_loss: {D_loss_avg:.4f}; G_loss: {G_loss_avg:.4f}')

    if G_loss_avg < best_g_loss:
        best_g_loss = G_loss_avg
        torch.save(G.state_dict(), os.path.join(save_dir, 'G_best.pth'))
        torch.save(D.state_dict(), os.path.join(save_dir, 'D_best.pth'))
        print(f"Saved Best Models at epoch {epoch} | G_loss: {best_g_loss:.4f}")

    save_sample(G, epoch, mb_size, Z_dim)


# Inference    
# G.load_state_dict(torch.load('checkpoints/G_best.pth'))
# G.eval()

# save_sample(G, "best", mb_size, Z_dim)

100%|██████████| 938/938 [00:07<00:00, 122.12it/s]


epoch0; D_loss: 0.0471; G_loss: 6.8148
Saved Best Models at epoch 0 | G_loss: 6.8148


100%|██████████| 938/938 [00:07<00:00, 120.08it/s]


epoch1; D_loss: 0.0135; G_loss: 7.4256


100%|██████████| 938/938 [00:07<00:00, 122.39it/s]


...

epoch70; D_loss: 0.6502; G_loss: 2.2794


100%|██████████| 938/938 [00:07<00:00, 123.32it/s]


epoch71; D_loss: 0.6512; G_loss: 2.2866


 80%|████████  | 755/938 [00:06<00:01, 121.62it/s]


KeyboardInterrupt: 